Import necessary packages

In [ ]:
from langchain_google_vertexai import VertexAI, VertexAIEmbeddings, ChatVertexAI
import vertexai
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_google_vertexai import ChatVertexAI
from pathlib import Path
from langchain_core.output_parsers import StrOutputParser
from typing import List, Sequence
from langgraph.graph import END, MessageGraph



In [17]:
#function to call a path to the file and read it. 
def context_gen(file_name):
    Folder = "VertexLang"
    here = Path(locals().get('__file__', Folder)).resolve()
    parameter = (here / file_name).read_text()
    return parameter

#remove code fences from the output
def remove_code_fences(text):
    lines = text.split("'")
    lines[0] = lines[0].replace(' -', '-', 1)
    return "'".join(lines)

#select an output parser
output_parser = StrOutputParser()

#function definition to test the vertexai model
#Intiate vertexai and the model that has been chosen for generation
vertexai.init(project = "totemic-veld-412608")
llm = ChatVertexAI(model_name = "gemini-1.0-pro", 
                temperature = "0.5", 
                max_output_tokens = 1000)



#create inputs to the model, telling it what needs to be done. 
system = context_gen("systemmsg.txt")
input_examples = context_gen("inputex.txt")
schema = context_gen("dataBaseSchema.txt")
human = ("select all boltholes and then select their corresponding edges and then select all flanges")
reflect = ChatPromptTemplate.from_messages([("human", ref_system), ("assistant", generated_response), ("user", query)])
chat = ChatVertexAI()

chain = reflect | chat | output_parser
result = chain.invoke({})
print(remove_code_fences(result))
print()


- filter: select * from faces where name like "%BoltHole%"
- expand: faceedges
- filter: select * from faces where name like "%Flange%"



Reflection prompt

In [19]:
ref_system = context_gen("ref_system.txt")
generated_response = result
query = human
prompt = ChatPromptTemplate.from_messages([("system", ref_system), ("user", input_examples), ("assistant", schema), ("human", human)])
chat = ChatVertexAI()
chain = reflect | chat | output_parser
reflected_response = chain.invoke({})
print(remove_code_fences(reflected_response))

- filter: select * from faces where name like "%BoltHole%"
- expand: faceedges
- filter: select * from faces where name like "%Flange%"
